In [1]:
import os,gc
os.environ["CUDA_VISIBLE_DEVICES"]="0"

import cudf

In [2]:
df = cudf.read_parquet('../../data/train_data/train.parquet')
df['hour'] = df.ts % (60*60*24)

df = df.loc[ (df.hour < 60*60*14)&(df.hour >= 60*60*2) ]

df['day'] = df.ts % (60*60*24*7)
print( df.shape )
df.head()

(71842047, 6)


,session,aid,ts,type,hour,day
30,0,1469630,1659531937,0,47137,565537
55,0,1725503,1659774028,0,30028,202828
56,0,528847,1659774232,0,30232,203032
57,0,1816325,1659774337,0,30337,203137
58,0,984597,1659774357,0,30357,203157


In [3]:
df = df.sort_values(['session','ts'])
df['prev'] = df.groupby('session').aid.diff(1)
df['next'] = df.groupby('session').aid.diff(-1)

df['order'] = (df.type==2).astype('int8')
df['cart'] = (df.type==1).astype('int8')

In [4]:
df.head()

,session,aid,ts,type,hour,day,prev,next,order,cart
30,0,1469630,1659531937,0,47137,565537,<NA>,-255873,0,0
55,0,1725503,1659774028,0,30028,202828,255873,1196656,0,0
56,0,528847,1659774232,0,30232,203032,-1196656,-1287478,0,0
57,0,1816325,1659774337,0,30337,203137,1287478,831728,0,0
58,0,984597,1659774357,0,30357,203157,-831728,-88185,0,0


In [5]:
df['n'] = df.groupby(['session','aid','type']).aid.transform('count')
tmp = df.loc[df['type']==2].drop_duplicates(['session','aid']).groupby('aid').n.mean()
tmp2 = df.loc[df['type']==1].drop_duplicates(['session','aid']).groupby('aid').n.mean()

In [6]:
tmp.head()

aid
632127      1.0
72178       1.0
897761      1.0
1325151     4.5
1028785    <NA>
Name: n, dtype: float64

In [7]:
tmp.max(), tmp2.max()

(375.0, 480.0)

In [8]:
item_features3 = df.groupby('aid').agg({'prev':'nunique','next':'nunique','order':'sum','cart':'sum'})
item_features3 = item_features3.merge(tmp,left_index=True, right_index=True, how='left')
item_features3.columns = ['prev3','next3','orders3','carts3','buy_count3']
item_features3 = item_features3.merge(tmp2,left_index=True, right_index=True, how='left')
item_features3.columns = ['prev3','next3','orders3','carts3','order_repeat3','cart_repeat3']
item_features3 = item_features3.fillna(-1)

i32 = ['prev3','next3','orders3','carts3']
for c in i32: item_features3[c] = item_features3[c].astype('int32')
    
f32 = ['order_repeat3','cart_repeat3']
for c in f32: item_features3[c] = item_features3[c].astype('float32')
    
del tmp, tmp2
gc.collect()

88

In [9]:
item_features3.head()

,prev3,next3,orders3,carts3,order_repeat3,cart_repeat3
aid,,,,,,
6495,51,52,17,40,1.125,1.823529
6482,11,10,0,1,-1.000,-1.000000
6490,12,11,0,1,-1.000,-1.000000
6479,2,2,0,0,-1.000,-1.000000
6480,10,9,0,0,-1.000,-1.000000


In [10]:
item_features = df.groupby('aid').agg({'type':'mean','aid':'count','session':'nunique','hour':'mean','day':'mean'})
item_features.columns = ['buy_ratio3','count_item3','count_user3','hour_mean3','day_mean3']

In [11]:
item_features.head()

,buy_ratio3,count_item3,count_user3,hour_mean3,day_mean3
aid,,,,,
0,0.000000,14,12,35580.571429,387352.000000
1,0.000000,15,13,34156.066667,218476.066667
2,0.000000,10,10,38617.300000,289177.300000
3,0.061620,568,349,36983.640845,321890.683099
4,0.046512,43,34,36887.813953,241836.651163


In [12]:
item_features2 = df.groupby('aid').agg({'type':'std','hour':'std','day':'std'}).fillna(-1)
item_features2.columns = ['buy_ratio_std3','hour_std3','day_std3']

f32 = ['buy_ratio_std3','hour_std3','day_std3']
for c in f32: item_features2[c] = item_features2[c].astype('float32')

In [13]:
item_features2.head()

,buy_ratio_std3,hour_std3,day_std3
aid,,,
487470,0.416270,8281.281250,133539.390625
1337516,0.000000,13519.512695,210987.609375
1497481,0.255609,6579.104980,140164.828125
201190,0.000000,7371.500977,183676.921875
732581,0.730996,10583.676758,180994.312500


In [14]:
item_features['repeat3'] = item_features.count_item3 / item_features.count_user3

f32 = ['buy_ratio3','hour_mean3','day_mean3','repeat3']
for c in f32: item_features[c] = item_features[c].astype('float32')

i32 = ['count_item3','count_user3']
for c in i32: item_features[c] = item_features[c].astype('int32')

In [15]:
item_features = cudf.concat([item_features,item_features2,item_features3],axis=1)

In [16]:
item_features.head()

,buy_ratio3,count_item3,count_user3,hour_mean3,day_mean3,repeat3,buy_ratio_std3,hour_std3,day_std3,prev3,next3,orders3,carts3,order_repeat3,cart_repeat3
aid,,,,,,,,,,,,,,,
0,0.000000,14,12,35580.570312,387352.00000,1.166667,0.000000,6677.466797,153575.203125,12,13,0,0,-1.0,-1.000000
1,0.000000,15,13,34156.066406,218476.06250,1.153846,0.000000,12305.671875,121742.343750,15,14,0,0,-1.0,-1.000000
2,0.000000,10,10,38617.300781,289177.31250,1.000000,0.000000,8129.087402,140863.578125,8,9,0,0,-1.0,-1.000000
3,0.061620,568,349,36983.640625,321890.68750,1.627507,0.274884,8935.369141,165253.578125,115,100,5,25,2.0,1.333333
4,0.046512,43,34,36887.812500,241836.65625,1.264706,0.213083,9552.584961,158150.156250,34,27,0,2,-1.0,1.000000


In [17]:
item_features.dtypes

buy_ratio3        float32
count_item3         int32
count_user3         int32
hour_mean3        float32
day_mean3         float32
repeat3           float32
buy_ratio_std3    float32
hour_std3         float32
day_std3          float32
prev3               int32
next3               int32
orders3             int32
carts3              int32
order_repeat3     float32
cart_repeat3      float32
dtype: object

In [18]:
COLS = [f'{x[:-1]}9' for x in item_features.columns]
item_features.columns = COLS
print(COLS)

['buy_ratio9', 'count_item9', 'count_user9', 'hour_mean9', 'day_mean9', 'repeat9', 'buy_ratio_std9', 'hour_std9', 'day_std9', 'prev9', 'next9', 'orders9', 'carts9', 'order_repeat9', 'cart_repeat9']


In [19]:
item_features.to_parquet('../../data/item_user_features/item22.pqt')